
실습 데이터 : https://openmv.net/info/travel-times

In [8]:
import pandas as pd

# 예제 데이터셋 불러오기 (인터넷에서 직접 다운로드)
url = 'https://openmv.net/file/travel-times.csv'
df = pd.read_csv(url)

print(df.head())
print(df.dtypes)


       Date StartTime  DayOfWeek GoingTo  Distance  MaxSpeed  AvgSpeed  \
0  1/6/2012     16:37     Friday    Home     51.29     127.4      78.3   
1  1/6/2012     08:20     Friday     GSK     51.63     130.3      81.8   
2  1/4/2012     16:17  Wednesday    Home     51.27     127.4      82.0   
3  1/4/2012     07:53  Wednesday     GSK     49.17     132.3      74.2   
4  1/3/2012     18:57    Tuesday    Home     51.15     136.2      83.4   

   AvgMovingSpeed FuelEconomy  TotalTime  MovingTime Take407All Comments  
0            84.8         NaN       39.3        36.3         No      NaN  
1            88.9         NaN       37.9        34.9         No      NaN  
2            85.8         NaN       37.5        35.9         No      NaN  
3            82.9         NaN       39.8        35.6         No      NaN  
4            88.1         NaN       36.8        34.8         No      NaN  
Date               object
StartTime          object
DayOfWeek          object
GoingTo            object
D

In [9]:
#FuelEconomy열 강제 형변환

df['FuelEconomy'] = pd.to_numeric(df['FuelEconomy'])

ValueError: Unable to parse string "-" at position 6

In [10]:
print(df['FuelEconomy'].unique()[:5])

[nan '-' '8.89' '9.08' '9.76']


In [11]:
import numpy as np
#df['FuelEconomy'] = df['FuelEconomy'].replace('-', np.nan)
df['FuelEconomy'] = pd.to_numeric(df['FuelEconomy'], errors='coerce')

# 변환 후 앞부분 확인
print(df['FuelEconomy'].head(10))
print("FuelEconomy dtype:", df['FuelEconomy'].dtype)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8    8.89
9    8.89
Name: FuelEconomy, dtype: float64
FuelEconomy dtype: float64


In [12]:
# Date 열을 datetime 형으로 변환
df['Date'] = pd.to_datetime(df['Date'])
print(df['Date'].head())

# 변환 후 확인 - Year, Month 열을 새로 만들어보기
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
print(df[['Date', 'Year', 'Month']].head(5))
print("Date dtype:", df['Date'].dtype)

0   2012-01-06
1   2012-01-06
2   2012-01-04
3   2012-01-04
4   2012-01-03
Name: Date, dtype: datetime64[ns]
        Date  Year  Month
0 2012-01-06  2012      1
1 2012-01-06  2012      1
2 2012-01-04  2012      1
3 2012-01-04  2012      1
4 2012-01-03  2012      1
Date dtype: datetime64[ns]


In [13]:
# YYYY-MM-DD 형식의 날짜를 가진 예시 데이터
df_example = pd.DataFrame({
    'date_str': ['2025-04-01', '2025-04-02', '2025-04-03']
})

# 기본 방법으로 변환 (자동 인식 가능하지만 명시적 지정도 가능)
df_example['date'] = pd.to_datetime(df_example['date_str'], format='%Y-%m-%d')

print(df_example)
print(df_example.dtypes)


     date_str       date
0  2025-04-01 2025-04-01
1  2025-04-02 2025-04-02
2  2025-04-03 2025-04-03
date_str            object
date        datetime64[ns]
dtype: object


In [14]:
# DayOfWeek 열을 범주형(category)으로 변환
df['DayOfWeek'] = df['DayOfWeek'].astype('category')

# 변환 후 dtype과 카테고리 목록 확인
print("DayOfWeek dtype:", df['DayOfWeek'].dtype)
print("Categories:", df['DayOfWeek'].cat.categories)


DayOfWeek dtype: category
Categories: Index(['Friday', 'Monday', 'Thursday', 'Tuesday', 'Wednesday'], dtype='object')


In [16]:
# 새로운 운행 기록 한 행 (dictionary로 생성)
new_trip = {
    'Date': '1/11/2012',
    'StartTime': '08:10',
    'DayOfWeek': 'Wednesday',
    'GoingTo': 'GSK',
    'Distance': 50.0,
    'MaxSpeed': 126.0,
    'AvgSpeed': 78.0,
    'AvgMovingSpeed': 82.0,
    'FuelEconomy': 7.9,
    'TotalTime': 42.0,
    'MovingTime': 40.0,
    'Take407All': 'No',
    'Comments': ''
}
new_df = pd.DataFrame([new_trip])  # 새로운 행을 가진 데이터프레임 생성

# 기존 df와 new_df를 연결하여 확장된 데이터프레임 만들기
df_extended = pd.concat([df, new_df], ignore_index=True)

# 연결 후 데이터프레임의 마지막 1행 확인
print(df_extended.tail(1))
print("연결 후 행 개수:", len(df_extended))


        Date StartTime  DayOfWeek GoingTo  Distance  MaxSpeed  AvgSpeed  \
0  1/11/2012     08:10  Wednesday     GSK      50.0     126.0      78.0   

   AvgMovingSpeed  FuelEconomy  TotalTime  MovingTime Take407All Comments  \
0            82.0          7.9       42.0        40.0         No            

   Year  Month  
0   NaN    NaN  
연결 후 행 개수: 206


In [17]:
# 목적지에 대한 추가 정보 데이터프레임
dest_info = pd.DataFrame({
    'GoingTo': ['Home', 'GSK', 'Cafe'],
    'DestinationType': ['Residence', 'Workplace', 'Other'],
    'Code': [1, 2, 3]
})
print(dest_info)


  GoingTo DestinationType  Code
0    Home       Residence     1
1     GSK       Workplace     2
2    Cafe           Other     3


In [18]:
# df와 dest_info를 GoingTo 키로 병합 (left join)
merged_df = pd.merge(df, dest_info, on='GoingTo', how='left')

# 병합 결과 확인 (일부 열만 출력)
print(merged_df[['Date', 'GoingTo', 'Distance', 'DestinationType', 'Code']].head(5))
print("병합 후 열 개수:", merged_df.shape[1])


        Date GoingTo  Distance DestinationType  Code
0 2012-01-06    Home     51.29       Residence     1
1 2012-01-06     GSK     51.63       Workplace     2
2 2012-01-04    Home     51.27       Residence     1
3 2012-01-04     GSK     49.17       Workplace     2
4 2012-01-03    Home     51.15       Residence     1
병합 후 열 개수: 17
